In [1]:
hh_in = r"J:\Shared drives\TMD_TSA\Model\software\PopSim\popsim16_real\output\synthetic_households.csv"
per_in = r"J:\Shared drives\TMD_TSA\Model\software\PopSim\popsim16_real\output\synthetic_persons.csv"
#blck_taz = r"J:\Shared drives\TMD_TSA\Model\platform\inputs\zonal\shp\taz_2010block_assignment.csv"
hh_per_out = r"J:\Shared drives\TMD_TSA\Model\software\PopSim\popsim16_real\output\PopulationSim_microhouseholds_2016_v20220630.csv"
blck_taz = r"C:\Projects\temp\taz_2010block_assignment.csv"

In [2]:
import pandas as pd

In [3]:
hh_df = pd.read_csv(hh_in)
per_df = pd.read_csv(per_in)

In [4]:
hh_df.head(), per_df.head()

(   household_id     PUMA   TAZ  HHNP  HHEMPCAT  HHINCPADJ
 0             1  2500100  6001     4         2     176578
 1             2  2500100  6001     2         1      44391
 2             3  2500100  6001     3         2     148069
 3             4  2500100  6001     2         0     191375
 4             5  2500100  6001     4         3      80890,
       PUMA   TAZ  household_id  EMP  PINCPADJ  AGEP
 0  2500100  6001             1    1    100620    51
 1  2500100  6001             1    1     75958    52
 2  2500100  6001             1    0         0    15
 3  2500100  6001             1    0    -19728    13
 4  2500100  6001             2    0     24662    74)

### Post Processing

1. Update attribute names
3. Calculate household attributes of workers, persons, and children
4. Attach household attributes to person table
5. zero out personal income for non-workers (and all negative incomes)
6. Assign a representative block
2. Create unique houseold and person ID


In [5]:
#Update attribute names
hh_df = hh_df.rename(columns={"household_id":"hid","HHINCPADJ":"hh_inc","TAZ":"taz_id"})
per_df = per_df.rename(columns={"household_id":"hid","PINCPADJ":"wage_inc","EMP":'is_worker',"AGEP":"age"})

# drop duplicate attributes
per_df = per_df.drop(columns={"PUMA","TAZ"})

hh_df, per_df

(             hid     PUMA  taz_id  HHNP  HHEMPCAT  hh_inc
 0              1  2500100    6001     4         2  176578
 1              2  2500100    6001     2         1   44391
 2              3  2500100    6001     3         2  148069
 3              4  2500100    6001     2         0  191375
 4              5  2500100    6001     4         3   80890
 ...          ...      ...     ...   ...       ...     ...
 3428254  3428255  4400400  208992     4         1   68247
 3428255  3428256  4400400  208992     2         1   45108
 3428256  3428257  4400400  208992     1         0   11252
 3428257  3428258  4400400  208992     1         0   11897
 3428258  3428259  4400400  208992     2         0   19552
 
 [3428259 rows x 6 columns],
              hid  is_worker  wage_inc  age
 0              1          1    100620   51
 1              1          1     75958   52
 2              1          0         0   15
 3              1          0    -19728   13
 4              2          0     24662   

In [6]:
#Calculate household attributes of workers, persons, and children

hh_per_df = pd.DataFrame(
        per_df.groupby('hid')['hid'].count()
        ).rename(columns={"hid":"persons"})

hh_chld_df = pd.DataFrame(
        per_df[per_df['age']<18].groupby('hid')['hid'].count()
        ).rename(columns={"hid":"children"})

hh_wrk_df = pd.DataFrame(
        per_df[per_df['is_worker']==1].groupby('hid')['hid'].count()
        ).rename(columns={"hid":"workers"})

hh_df = hh_df.merge(
    hh_per_df, on="hid", how="left").merge(
    hh_chld_df, on="hid", how="left").merge(
    hh_wrk_df, on="hid", how="left").fillna(0)

hh_df

#hh_per_df, hh_chld_df, hh_wrk_df

,hid,PUMA,taz_id,HHNP,HHEMPCAT,hh_inc,persons,children,workers
0,1,2500100,6001,4,2,176578,4,2.0,2.0
1,2,2500100,6001,2,1,44391,2,0.0,1.0
2,3,2500100,6001,3,2,148069,3,1.0,2.0
3,4,2500100,6001,2,0,191375,2,0.0,0.0
4,5,2500100,6001,4,3,80890,4,0.0,3.0
...,...,...,...,...,...,...,...,...,...
3428254,3428255,4400400,208992,4,1,68247,7,5.0,1.0
3428255,3428256,4400400,208992,2,1,45108,2,0.0,1.0
3428256,3428257,4400400,208992,1,0,11252,1,0.0,0.0
3428257,3428258,4400400,208992,1,0,11897,1,0.0,0.0


In [7]:
#Attach household attributes to person table
hh_per_df = per_df.merge(hh_df, on="hid")
hh_per_df

,hid,is_worker,wage_inc,age,PUMA,taz_id,HHNP,HHEMPCAT,hh_inc,persons,children,workers
0,1,1,100620,51,2500100,6001,4,2,176578,4,2.0,2.0
1,1,1,75958,52,2500100,6001,4,2,176578,4,2.0,2.0
2,1,0,0,15,2500100,6001,4,2,176578,4,2.0,2.0
3,1,0,-19728,13,2500100,6001,4,2,176578,4,2.0,2.0
4,2,0,24662,74,2500100,6001,2,1,44391,2,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8207748,3428256,1,6271,28,4400400,208992,2,1,45108,2,0.0,1.0
8207749,3428257,0,11252,58,4400400,208992,1,0,11252,1,0.0,0.0
8207750,3428258,0,11897,69,4400400,208992,1,0,11897,1,0.0,0.0
8207751,3428259,0,19552,62,4400400,208992,2,0,19552,2,0.0,0.0


In [8]:
#zero out personal income for non-workers (and all negative incomes)
hh_per_df.loc[hh_per_df['is_worker']==0,'wage_inc']=0
hh_per_df.loc[hh_per_df['wage_inc']<0,'wage_inc']=0
hh_per_df['wage_inc'].describe()

count    8.207753e+06
mean     3.078778e+04
std      5.828316e+04
min      0.000000e+00
25%      0.000000e+00
50%      1.926000e+03
75%      4.520200e+04
max      1.424434e+06
Name: wage_inc, dtype: float64

In [9]:
#Assign a representative block
blk_lu = pd.read_csv(blck_taz)
taz_lu = blk_lu.drop_duplicates(subset="taz_id")

hh_per_df = hh_per_df.merge(taz_lu,on="taz_id", how="left")
hh_per_df

,hid,is_worker,wage_inc,age,PUMA,taz_id,HHNP,HHEMPCAT,hh_inc,persons,children,workers,block_id
0,1,1,100620,51,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000
1,1,1,75958,52,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000
2,1,0,0,15,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000
3,1,0,0,13,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000
4,2,0,0,74,2500100,6001,2,1,44391,2,0.0,1.0,250039241001000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8207748,3428256,1,6271,28,4400400,208992,2,1,45108,2,0.0,1.0,208992
8207749,3428257,0,0,58,4400400,208992,1,0,11252,1,0.0,0.0,208992
8207750,3428258,0,0,69,4400400,208992,1,0,11897,1,0.0,0.0,208992
8207751,3428259,0,0,62,4400400,208992,2,0,19552,2,0.0,0.0,208992


In [10]:
#Create unique houseold and person ID
hh_per_df['hid'] = hh_per_df['block_id'].astype(str) + "_" + hh_per_df['hid'].astype(str)
hh_per_df['person_num'] = hh_per_df.groupby('hid').cumcount() + 1
hh_per_df

,hid,is_worker,wage_inc,age,PUMA,taz_id,HHNP,HHEMPCAT,hh_inc,persons,children,workers,block_id,person_num
0,250039241001000_1,1,100620,51,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000,1
1,250039241001000_1,1,75958,52,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000,2
2,250039241001000_1,0,0,15,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000,3
3,250039241001000_1,0,0,13,2500100,6001,4,2,176578,4,2.0,2.0,250039241001000,4
4,250039241001000_2,0,0,74,2500100,6001,2,1,44391,2,0.0,1.0,250039241001000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8207748,208992_3428256,1,6271,28,4400400,208992,2,1,45108,2,0.0,1.0,208992,2
8207749,208992_3428257,0,0,58,4400400,208992,1,0,11252,1,0.0,0.0,208992,1
8207750,208992_3428258,0,0,69,4400400,208992,1,0,11897,1,0.0,0.0,208992,1
8207751,208992_3428259,0,0,62,4400400,208992,2,0,19552,2,0.0,0.0,208992,1


In [11]:
# clean and export
hh_per_df[['block_id',
           'hid',
           'hh_inc',
           'persons',
           'workers',
           'children',
           'person_num',
           'age',
           'is_worker',
           'wage_inc']].to_csv(hh_per_out, index=False)